# Modelos de Regressão

Este notebook investiga diferentes abordagens de regressão para análise de dados, com foco nos modelos de Regressão Linear e Lasso. São aplicadas técnicas de pré-processamento, como a transformação de Box-Cox, visando aprimorar a modelagem. A avaliação dos modelos é feita com base em métricas como RMSE e R², considerando tanto o conjunto completo de dados quanto versões filtradas após a remoção de outliers das questões do Enem (vetorizadas).

## Apoio


In [2]:
# Importando Dependências para Modelos de Regressão
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import statsmodels.api as sm

from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error
from sklearn.linear_model import LassoCV, Lasso
from sklearn.model_selection import train_test_split

In [31]:
# Leitura do dataset com os embeddings de 300 dimensões
df_enem_300 = pd.read_pickle('../data/final/enem_data_embeddings_300.pkl')
df_enem_300.head()

,numero_questao,gabarito,NU_PARAM_A,nu_param_B,NU_PARAM_C,ANO,enunciado,alternativas,gabarito_texto,distratores,enunciado_limpo,alternativas_limpo,gabarito_texto_limpo,distratores_limpo,enunciado_embeddings_word2vec,gabarito_embeddings_word2vec
0,1,C,3.43894,0.97831,0.10855,2017,"No império africano do Mali, no século XIV, To...",A: isolamento geográﬁco do Saara ocidental; B...,posição relativa nas redes de circulação,isolamento geográﬁco do Saara ocidental; explo...,africano além astronomia centro cidade comérci...,A: isolamento ocidental saara; B: exploração i...,circulação posição redes relativa,competição econômica exploração intensiva isol...,"[-0.017395772, -0.004402813, -0.046908338, 0.0...","[-0.16301225, -0.142375, 0.012171499, -0.00457..."
1,2,D,3.00837,0.49169,0.13877,2017,Após a Declaração Universal dos Direitos Human...,A: ataque feito pelos japoneses à base milita...,execução de judeus e eslavos presos em guetos ...,ataque feito pelos japoneses à base militar am...,aberrações acontecimentos após assumida branco...,A: americana ataque base feito harbor japonese...,campos concentração eslavos execução guetos ju...,americana americanas ataque atômicas base bomb...,"[-0.051949076, 0.03128451, -0.072241075, 0.028...","[-0.18672037, 0.04755275, -0.17406088, -0.0398..."
2,3,D,0.60432,3.25992,0.08798,2017,"A moralidade, Bentham exortava, não é uma ques...",A: fundamentação cientíﬁca de viés positivist...,racionalidade de caráter pragmático,fundamentação cientíﬁca de viés positivista; c...,abstratas afetados agradar ação bentham condut...,A: fundamentação positivista viés; B: convençã...,caráter pragmático racionalidade,comportamental convenção fundamentação inclina...,"[0.026049094, 0.043251127, -0.029757027, -0.00...","[0.011740998, 0.07592634, -0.016144669, 0.1384..."
3,4,E,1.85031,0.57925,0.11344,2017,Fala-se muito nos dias de hoje em direitos do ...,A: modernização da educação escolar; B: atuali...,universalização do princípio da igualdade civil,modernização da educação escolar; atualização ...,apresenta assembleia bases bem cidadão concepç...,A: educação escolar modernização; B: atualizaç...,civil igualdade princípio universalização,aristocráticos atualização conhecimento costum...,"[-0.005251272, 0.039871164, -0.018038195, 0.06...","[-0.073669255, 0.12609875, 0.063628, 0.1261957..."
4,5,C,2.46290,0.76307,0.17672,2017,Na Constituição da República Federativa do Bra...,A: etnia e miscigenação racial; B: sociedade...,espaço e sobrevivência cultural,etnia e miscigenação racial; sociedade e igual...,aplicação artigo bens brasil competindo consta...,A: etnia miscigenação racial; B: igualdade jur...,cultural espaço sobrevivência,ambiental bem econômica educação etnia igualda...,"[0.01738678, 0.0033733728, -0.027247325, 0.002...","[0.11709199, -0.16062833, 0.082869664, -0.1195..."


In [32]:
# Leitura do dataset com os embeddings de 50 dimensões
df_enem_50 = pd.read_pickle('../data/final/enem_data_embeddings_50.pkl')
df_enem_50.head()

,numero_questao,gabarito,NU_PARAM_A,nu_param_B,NU_PARAM_C,ANO,enunciado,alternativas,gabarito_texto,distratores,enunciado_limpo,alternativas_limpo,gabarito_texto_limpo,distratores_limpo,enunciado_embeddings_word2vec,gabarito_embeddings_word2vec
0,1,C,3.43894,0.97831,0.10855,2017,"No império africano do Mali, no século XIV, To...",A: isolamento geográﬁco do Saara ocidental; B...,posição relativa nas redes de circulação,isolamento geográﬁco do Saara ocidental; explo...,africano além astronomia centro cidade comérci...,A: isolamento ocidental saara; B: exploração i...,circulação posição redes relativa,competição econômica exploração intensiva isol...,"[0.09772969, 0.013166125, -0.032748703, 0.0580...","[-0.04434625, 0.2105245, -0.18144351, 0.030336..."
1,2,D,3.00837,0.49169,0.13877,2017,Após a Declaração Universal dos Direitos Human...,A: ataque feito pelos japoneses à base milita...,execução de judeus e eslavos presos em guetos ...,ataque feito pelos japoneses à base militar am...,aberrações acontecimentos após assumida branco...,A: americana ataque base feito harbor japonese...,campos concentração eslavos execução guetos ju...,americana americanas ataque atômicas base bomb...,"[-0.018823402, 0.07832528, -0.016192827, 0.027...","[0.21008351, 0.5248949, 0.03625112, 0.01772637..."
2,3,D,0.60432,3.25992,0.08798,2017,"A moralidade, Bentham exortava, não é uma ques...",A: fundamentação cientíﬁca de viés positivist...,racionalidade de caráter pragmático,fundamentação cientíﬁca de viés positivista; c...,abstratas afetados agradar ação bentham condut...,A: fundamentação positivista viés; B: convençã...,caráter pragmático racionalidade,comportamental convenção fundamentação inclina...,"[-0.013062097, 0.038470805, -0.020605344, 0.05...","[0.16413634, -0.035050992, 0.08454134, 0.23227..."
3,4,E,1.85031,0.57925,0.11344,2017,Fala-se muito nos dias de hoje em direitos do ...,A: modernização da educação escolar; B: atuali...,universalização do princípio da igualdade civil,modernização da educação escolar; atualização ...,apresenta assembleia bases bem cidadão concepç...,A: educação escolar modernização; B: atualizaç...,civil igualdade princípio universalização,aristocráticos atualização conhecimento costum...,"[-0.042833664, -0.01434522, -0.002431296, 0.01...","[0.102909006, 0.06116425, -0.13846825, 0.33034..."
4,5,C,2.46290,0.76307,0.17672,2017,Na Constituição da República Federativa do Bra...,A: etnia e miscigenação racial; B: sociedade...,espaço e sobrevivência cultural,etnia e miscigenação racial; sociedade e igual...,aplicação artigo bens brasil competindo consta...,A: etnia miscigenação racial; B: igualdade jur...,cultural espaço sobrevivência,ambiental bem econômica educação etnia igualda...,"[0.010493249, 0.11095436, 0.017243173, 0.05319...","[0.267512, -0.081234336, 0.08019033, 0.1294613..."


In [59]:
# Leitura do dataset com os embeddings de 100 dimensões
df_enem_100 = pd.read_pickle('../data/final/enem_data_embeddings_100.pkl')
df_enem_100.head()

,numero_questao,gabarito,NU_PARAM_A,nu_param_B,NU_PARAM_C,ANO,enunciado,alternativas,gabarito_texto,distratores,enunciado_limpo,alternativas_limpo,gabarito_texto_limpo,distratores_limpo,enunciado_embeddings_word2vec,gabarito_embeddings_word2vec
0,1,C,3.43894,0.97831,0.10855,2017,"No império africano do Mali, no século XIV, To...",A: isolamento geográﬁco do Saara ocidental; B...,posição relativa nas redes de circulação,isolamento geográﬁco do Saara ocidental; explo...,africano além astronomia centro cidade comérci...,A: isolamento ocidental saara; B: exploração i...,circulação posição redes relativa,competição econômica exploração intensiva isol...,"[0.061125945, -0.048051268, -0.059038285, 0.05...","[0.0875405, 0.14673425, 0.17936975, -0.0332287..."
1,2,D,3.00837,0.49169,0.13877,2017,Após a Declaração Universal dos Direitos Human...,A: ataque feito pelos japoneses à base milita...,execução de judeus e eslavos presos em guetos ...,ataque feito pelos japoneses à base militar am...,aberrações acontecimentos após assumida branco...,A: americana ataque base feito harbor japonese...,campos concentração eslavos execução guetos ju...,americana americanas ataque atômicas base bomb...,"[0.009677129, 0.11954352, 0.021252066, -0.0227...","[0.16129163, 0.23760912, 0.080258876, -0.05254..."
2,3,D,0.60432,3.25992,0.08798,2017,"A moralidade, Bentham exortava, não é uma ques...",A: fundamentação cientíﬁca de viés positivist...,racionalidade de caráter pragmático,fundamentação cientíﬁca de viés positivista; c...,abstratas afetados agradar ação bentham condut...,A: fundamentação positivista viés; B: convençã...,caráter pragmático racionalidade,comportamental convenção fundamentação inclina...,"[0.068149306, -0.023954527, 0.02739678, -0.003...","[0.26195732, -0.13758467, -0.16108766, 0.07133..."
3,4,E,1.85031,0.57925,0.11344,2017,Fala-se muito nos dias de hoje em direitos do ...,A: modernização da educação escolar; B: atuali...,universalização do princípio da igualdade civil,modernização da educação escolar; atualização ...,apresenta assembleia bases bem cidadão concepç...,A: educação escolar modernização; B: atualizaç...,civil igualdade princípio universalização,aristocráticos atualização conhecimento costum...,"[0.019322807, -0.034913678, -0.00591799, 0.012...","[0.253879, -0.005192494, -0.028254755, 0.10982..."
4,5,C,2.46290,0.76307,0.17672,2017,Na Constituição da República Federativa do Bra...,A: etnia e miscigenação racial; B: sociedade...,espaço e sobrevivência cultural,etnia e miscigenação racial; sociedade e igual...,aplicação artigo bens brasil competindo consta...,A: etnia miscigenação racial; B: igualdade jur...,cultural espaço sobrevivência,ambiental bem econômica educação etnia igualda...,"[-0.032498874, 0.08808657, -0.040701427, 0.077...","[0.248065, -0.15562867, -0.114118, -0.12127533..."


In [60]:
# Removendo observações em que o parâmetro B é nulo
df_enem_300 = df_enem_300[df_enem_300['nu_param_B'].notna()]
df_enem_50 = df_enem_50[df_enem_50['nu_param_B'].notna()]
df_enem_100 = df_enem_100[df_enem_100['nu_param_B'].notna()]

## Regressão Linear

### 300 dimensões

In [34]:
# Separando os dados
X = [np.array(embedding) for embedding in df_enem_300['enunciado_embeddings_word2vec']]
y = df_enem_300['nu_param_B']

In [35]:
# Aplicando Transformações
add_list = [(y.min() * (-1)) + 1] * len(y)
y = y + add_list

# Aplicando Boxcox
y, _ = stats.boxcox(y)

In [36]:
# Separando em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [37]:
# Adicionando constante
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

In [38]:
model = sm.OLS(y_train, X_train).fit()
print(model.summary(alpha=0.05))

/home/alesouza/.cache/pypoetry/virtualenvs/projeto-enem-YISx0mk8-py3.11/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/home/alesouza/.cache/pypoetry/virtualenvs/projeto-enem-YISx0mk8-py3.11/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: invalid value encountered in scalar multiply
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/home/alesouza/.cache/pypoetry/virtualenvs/projeto-enem-YISx0mk8-py3.11/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1717: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Tue, 13 May 2025   Prob (F-statistic):                nan
Time:                        10:56:50   Log-Likelihood:                 5970.9
No. Observations:                 186   AIC:                        -1.157e+04
Df Residuals:                       0   BIC:                        -1.097e+04
Df Model:                         185                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.2107        inf          0        n

In [39]:
# Calculando o RMSE
pred = model.predict(X_test)

rms = root_mean_squared_error(y_test, pred)

print("RMSE com todas as variáveis:", rms)

RMSE com todas as variáveis: 0.7720791146717553


### 50 dimensões

In [40]:
# Separando os dados
X = [np.array(embedding) for embedding in df_enem_50['enunciado_embeddings_word2vec']]
y = df_enem_50['nu_param_B']

In [41]:
# Aplicando Transformações
add_list = [(y.min() * (-1)) + 1] * len(y)
y = y + add_list

# Aplicando Boxcox
y, _ = stats.boxcox(y)

In [42]:
# Separando em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [43]:
# Adicionando constante
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

In [44]:
model = sm.OLS(y_train, X_train).fit()
print(model.summary(alpha=0.05))

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.316
Model:                            OLS   Adj. R-squared:                  0.062
Method:                 Least Squares   F-statistic:                     1.245
Date:                Tue, 13 May 2025   Prob (F-statistic):              0.162
Time:                        10:57:52   Log-Likelihood:                -93.299
No. Observations:                 186   AIC:                             288.6
Df Residuals:                     135   BIC:                             453.1
Df Model:                          50                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.5116      0.659      2.292      0.0

In [45]:
# Calculando o RMSE
pred = model.predict(X_test)

rms = root_mean_squared_error(y_test, pred)

print("RMSE com todas as variáveis:", rms)

RMSE com todas as variáveis: 0.5394759971562638


### 100 dimensões

In [61]:
# Separando os dados
X = [np.array(embedding) for embedding in df_enem_100['enunciado_embeddings_word2vec']]
y = df_enem_100['nu_param_B']

In [62]:
# Aplicando Transformações
add_list = [(y.min() * (-1)) + 1] * len(y)
y = y + add_list

# Aplicando Boxcox
y, _ = stats.boxcox(y)

In [63]:
# Separando em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [64]:
# Adicionando constante
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

In [65]:
model = sm.OLS(y_train, X_train).fit()
print(model.summary(alpha=0.05))

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.625
Model:                            OLS   Adj. R-squared:                  0.184
Method:                 Least Squares   F-statistic:                     1.417
Date:                Tue, 13 May 2025   Prob (F-statistic):             0.0498
Time:                        11:14:58   Log-Likelihood:                -37.361
No. Observations:                 186   AIC:                             276.7
Df Residuals:                      85   BIC:                             602.5
Df Model:                         100                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.9890      0.915      2.173      0.0

In [66]:
# Calculando o RMSE
pred = model.predict(X_test)

rms = root_mean_squared_error(y_test, pred)

print("RMSE com todas as variáveis:", rms)

RMSE com todas as variáveis: 0.7525703952360024


## Regressão Lasso

### 300 dimensões

In [46]:
# Separando os dados
X = [np.array(embedding) for embedding in df_enem_300['enunciado_embeddings_word2vec']]
y = df_enem_300['nu_param_B']

In [47]:
# Aplicando Transformações
add_list = [(y.min() * (-1)) + 1] * len(y)
y = y + add_list

# Aplicando Boxcox
y, _ = stats.boxcox(y)

In [48]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [49]:
model_lasso = LassoCV(alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10], random_state=42).fit(
    X_train, y_train
)
pred_lasso = model_lasso.predict(X_test)

print("melhor r2 score (treino):", model_lasso.score(X_train, y_train))
print("r2 score (teste):", model_lasso.score(X_test, y_test))
print("melhor alpha:", model_lasso.alpha_)
print("RMSE com o alpha escolhido:", root_mean_squared_error(y_test, pred_lasso))

melhor r2 score (treino): 0.5344153222227536
r2 score (teste): -0.19432811792646487
melhor alpha: 0.001
RMSE com o alpha escolhido: 0.5177163330655292


#### Modelo com corte na dificuldade (-3, 3)

In [50]:
enem_filtered = df_enem_300.copy()
enem_filtered = enem_filtered[
    (enem_filtered["nu_param_B"] >= -3) & (enem_filtered["nu_param_B"] <= 3)
]
enem_filtered["nu_param_B"].describe()

count    263.000000
mean       1.075595
std        0.661348
min       -0.726450
25%        0.612925
50%        1.051180
75%        1.535350
max        2.802770
Name: nu_param_B, dtype: float64

In [51]:
# Separando os dados
X = [np.array(embedding) for embedding in enem_filtered['enunciado_embeddings_word2vec']]
y = enem_filtered['nu_param_B']

In [52]:
# Aplicando Transformações
add_list = [(y.min() * (-1)) + 1] * len(y)
y = y + add_list

# Aplicando Boxcox
y, _ = stats.boxcox(y)

In [53]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [54]:
model_lasso = LassoCV(alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10, 100], random_state=42).fit(
    X_train, y_train
)
pred_lasso = model_lasso.predict(X_test)

print("melhor r2 score (treino):", model_lasso.score(X_train, y_train))
print("r2 score (teste):", model_lasso.score(X_test, y_test))
print("melhor alpha:", model_lasso.alpha_)
print("RMSE com o alpha escolhido:", root_mean_squared_error(y_test, pred_lasso))

melhor r2 score (treino): 0.0
r2 score (teste): -0.01961968855705387
melhor alpha: 100.0
RMSE com o alpha escolhido: 0.608577671186741


### 50 dimensões

In [55]:
# Separando os dados
X = [np.array(embedding) for embedding in df_enem_50['enunciado_embeddings_word2vec']]
y = df_enem_50['nu_param_B']

In [56]:
# Aplicando Transformações
add_list = [(y.min() * (-1)) + 1] * len(y)
y = y + add_list

# Aplicando Boxcox
y, _ = stats.boxcox(y)

In [57]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [58]:
model_lasso = LassoCV(alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10], random_state=42).fit(
    X_train, y_train
)
pred_lasso = model_lasso.predict(X_test)

print("melhor r2 score (treino):", model_lasso.score(X_train, y_train))
print("r2 score (teste):", model_lasso.score(X_test, y_test))
print("melhor alpha:", model_lasso.alpha_)
print("RMSE com o alpha escolhido:", root_mean_squared_error(y_test, pred_lasso))

melhor r2 score (treino): 0.0
r2 score (teste): -0.08576553465012338
melhor alpha: 10.0
RMSE com o alpha escolhido: 0.49362604239339286


### 100 dimensões

In [68]:
# Separando os dados
X = [np.array(embedding) for embedding in df_enem_100['enunciado_embeddings_word2vec']]
y = df_enem_100['nu_param_B']

In [69]:
# Aplicando Transformações
add_list = [(y.min() * (-1)) + 1] * len(y)
y = y + add_list

# Aplicando Boxcox
y, _ = stats.boxcox(y)

In [70]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [71]:
model_lasso = LassoCV(alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10], random_state=42).fit(
    X_train, y_train
)
pred_lasso = model_lasso.predict(X_test)

print("melhor r2 score (treino):", model_lasso.score(X_train, y_train))
print("r2 score (teste):", model_lasso.score(X_test, y_test))
print("melhor alpha:", model_lasso.alpha_)
print("RMSE com o alpha escolhido:", root_mean_squared_error(y_test, pred_lasso))

melhor r2 score (treino): 0.0
r2 score (teste): -0.08576553465012338
melhor alpha: 10.0
RMSE com o alpha escolhido: 0.49362604239339286
